In [4]:
import os
import io
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Image as IPImage
from datetime import datetime
from scripts.utils import load_scenario_data
from src.environment import AircraftDisruptionOptimizer
from scripts.visualizations import StatePlotter

def run_exact_solution_and_plot(scenario_folder):
    # Load scenario data
    data_dict = load_scenario_data(scenario_folder)
    aircraft_dict = data_dict['aircraft']
    flights_dict = data_dict['flights']
    rotations_dict = data_dict['rotations']
    alt_aircraft_dict = data_dict['alt_aircraft']
    config_dict = data_dict['config']

    # Parse recovery period times
    recovery_period = config_dict['RecoveryPeriod']
    start_datetime = datetime.strptime(
        f"{recovery_period['StartDate']} {recovery_period['StartTime']}", '%d/%m/%y %H:%M'
    )
    end_datetime = datetime.strptime(
        f"{recovery_period['EndDate']} {recovery_period['EndTime']}", '%d/%m/%y %H:%M'
    )

    # Initialize optimizer
    optimizer = AircraftDisruptionOptimizer(
        aircraft_dict=aircraft_dict,
        flights_dict=flights_dict,
        rotations_dict=rotations_dict,
        alt_aircraft_dict=alt_aircraft_dict,
        config_dict=config_dict
    )

    # Initialize state plotter
    state_plotter = StatePlotter(
        aircraft_dict=aircraft_dict,
        flights_dict=flights_dict,
        rotations_dict=rotations_dict,
        alt_aircraft_dict=alt_aircraft_dict,
        start_datetime=start_datetime,
        end_datetime=end_datetime
    )

    # List to store plots
    plots = []

    # Plot initial state
    fig = state_plotter.plot_state(
        flights_dict,
        swapped_flights=[],
        environment_delayed_flights=set(),
        cancelled_flights=set(),
        current_datetime=start_datetime,
        title_appendix="Initial State",
        show_plot=False
    )
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img = IPImage(data=buf.read(), format='png', embed=True)
    plots.append(img)
    plt.close(fig)

    # Solve the optimization problem
    solution = optimizer.solve()

    # Extract solution details
    swapped_flights = [(f_id, new_ac) for f_id, new_ac in solution['assignments'].items()]
    environment_delayed_flights = set(solution['delays'].keys())
    cancelled_flights = set(solution['cancellations'])

    # Update flights dict with delays
    updated_flights_dict = flights_dict.copy()
    for flight_id, delay in solution['delays'].items():
        if flight_id in updated_flights_dict:
            flight_info = updated_flights_dict[flight_id]
            flight_info['Delay'] = delay

    # Plot final (solution) state
    fig = state_plotter.plot_state(
        updated_flights_dict,
        swapped_flights,
        environment_delayed_flights,
        cancelled_flights,
        current_datetime=end_datetime,
        title_appendix="Optimizer Solution",
        show_plot=False
    )
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img = IPImage(data=buf.read(), format='png', embed=True)
    plots.append(img)
    plt.close(fig)

    # Create widgets to navigate through the plots
    output = widgets.Output()

    def update_plot(index):
        with output:
            clear_output(wait=True)
            display(plots[index])

    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(plots)-1,
        step=1,
        description='Step:'
    )

    def on_slider_change(change):
        if change['name'] == 'value':
            update_plot(change['new'])

    slider.observe(on_slider_change, names='value')

    prev_button = widgets.Button(description='⬅️', layout=widgets.Layout(width='40px'))
    next_button = widgets.Button(description='➡️', layout=widgets.Layout(width='40px'))

    def on_prev_button_clicked(b):
        slider.value = max(0, slider.value - 1)

    def on_next_button_clicked(b):
        slider.value = min(len(plots)-1, slider.value + 1)

    prev_button.on_click(on_prev_button_clicked)
    next_button.on_click(on_next_button_clicked)

    navigation = widgets.HBox([prev_button, next_button, slider])
    update_plot(0)
    display(navigation, output)

    # Print out solution statistics
    print("\nOptimization Results:")
    print(f"Objective value: {solution['objective_value']:.2f}")
    print(f"\nSolution statistics:")
    print(f"  Runtime: {solution['statistics']['runtime']:.2f} seconds")
    print(f"  Status: {solution['statistics']['status']}")
    print(f"\nSolution summary:")
    print(f"  Cancelled flights: {len(solution['cancellations'])}")
    print(f"  Total delay minutes: {solution['total_delay_minutes']}")
    print(f"  Number of reassignments: {len(solution['assignments'])}")

    if solution['cancellations']:
        print("\nCancelled flights:")
        for flight_id in solution['cancellations']:
            print(f"  - Flight {flight_id}")

    if solution['delays']:
        print("\nDelayed flights:")
        for flight_id, delay in solution['delays'].items():
            print(f"  - Flight {flight_id}: {delay} minutes")

    if solution['assignments']:
        print("\nFlight reassignments:")
        for flight_id, new_aircraft in solution['assignments'].items():
            original_aircraft = rotations_dict[flight_id]['Aircraft']
            if new_aircraft != original_aircraft:
                print(f"  - Flight {flight_id}: {original_aircraft} -> {new_aircraft}")


scenario_folder = "../data/Training/6ac-700-diverse/mixed_high_Scenario_004"
run_exact_solution_and_plot(scenario_folder)



Step 1:
Current conflicts: {('B737#1', 2.0, 389.0, 582.0), ('B737#2', 6.0, 618.0, 913.0), ('B737#2', 4.0, 165.0, 377.0), ('B737#1', 3.0, 592.0, 857.0), ('B737#2', 5.0, 393.0, 593.0)}

Evaluating 120 valid actions:
  Action 0 (flight=0, aircraft=0): reward=-60.0
    -> New best action (reward=-60.0)
  Action 7 (flight=1, aircraft=0): reward=-5055.5
  Action 8 (flight=1, aircraft=1): reward=-60.0
  Action 9 (flight=1, aircraft=2): reward=-2555.5
  Action 10 (flight=1, aircraft=3): reward=-2555.5
  Action 11 (flight=1, aircraft=4): reward=-2555.5
  Action 12 (flight=1, aircraft=5): reward=-2555.5
  Action 13 (flight=1, aircraft=6): reward=-55.5
    -> New best action (reward=-55.5)
  Action 14 (flight=2, aircraft=0): reward=-5021.1
  Action 15 (flight=2, aircraft=1): reward=-2521.1
  Action 16 (flight=2, aircraft=2): reward=-2521.1
  Action 17 (flight=2, aircraft=3): reward=-2521.1
  Action 18 (flight=2, aircraft=4): reward=-2521.1
  Action 19 (flight=2, aircraft=5): reward=-1585.1
  Act

Output()


Optimization Results:
Objective value: 8824.20

Solution statistics:
  Runtime: 0.67 seconds
  Status: Complete

Solution summary:
  Cancelled flights: 0
  Total delay minutes: 337.0
  Number of reassignments: 0

Delayed flights:
  - Flight 3: 49.0 minutes
  - Flight 6.0: 288.0 minutes
